간단하게 실험용 Cluster를 구축 할때 아래와 같이 Master 서버가 Slave 서버가 외부 네트워크와 통신할 수 있도록 IP 공유기 역할을 해야 할때 IP MASQUERADE를 이용하는 방법입니다.

```
Internet <--> Master <--> 스위치허브 <---> Slave1,2,3 ...
```




## Master 서버

### interface 설정
Master 서버는 랜카드를 2개 가지고 있다고 가정합니다.
eth0은 외부 네트워크용이고 eth1은 내부 내트워크용 랜카드 입니다.

```
> sudo vi /etc/network/inetrfaces
```
설정 파일을 열어보면 eth0은 보통 인터넷 공유기와 바로 연결되어 있어 dhcp 로 설정되어 있을 것입니다.

이제 아래와 같이 eth1 랜카드를 내부 네트워크 고정 ip 로 설정합니다.

```
# interfaces(5) file used by ifup(8) and ifdown(8)
auto lo
iface lo inet loopback

auto eth0
iface eth0 inet dhcp

auto eth1
iface eth1 inet static
address 192.168.0.1
netmask 255.255.255.0
network 192.168.0.0
broadcast 192.168.10.255
gateway 192.168.0.1
dns-nameservers 8.8.8.8
```
설정 후 네트워크를 재시작합니다.

```
> sudo service networking restart
```

### dns 서버 설정
resolvconf 서비스를 통해서 접근할 dns 서버를 설정해 줍니다.

```
> sudo vi /etc/resolvconf/resolv.conf.d/head
```
설정 파일을 열어 아래와 같이 수정합니다.

```
# Dynamic resolv.conf(5) file for glibc resolver(3) generated by resolvconf(8)
#     DO NOT EDIT THIS FILE BY HAND -- YOUR CHANGES WILL BE OVERWRITTEN
nameserver 8.8.8.8
```
설정 파일 저장 후 resolvconf 서비스를 재시작합니다.

```
> sudo service resolvconf retart
```

## iptables 설정
이제 Master 서버의 eth1 내부 네트워크를 통해 요청되는 패킷을 eth0 외부 네트워크카드로 전달하도록 해주는 rule 를 iptables 라는 명령어를 통해 등록해 줍니다.

```
> sudo /sbin/iptables -A FORWARD -o etho -j ACCEPT
> sudo /sbin/iptables -t nat -A POSTROUTING -o etho -j MASQUERADE
```

iptables rule 은 재부팅시에 사라지게 되므로 재부팅 시 마다 자동으로 rule 이 등록 될 수 있도록 /etc/rc.local 파일을 열어 등록해줍니다.

```
> sudo vi /etc/rc.local
```
아래와 같이 명령을 등록해 줍니다.

```
#!/bin/sh -e
#
# rc.local
#
# This script is executed at the end of each multiuser runlevel.
# Make sure that the script will "exit 0" on success or any other
# value on error.
#
# In order to enable or disable this script just change the execution
# bits.
#
# By default this script does nothing.

/sbin/iptables -A FORWARD -o eth0 -j ACCEPT
/sbin/iptables -t nat -A POSTROUTING -o eth0 -j MASQUERADE

exit 0
```

## Slave 서버 설정
Slave 서버의 네트워크 interface 와 dns 서버 설정은  Master 서버의 내부 네트워크 interface 설정하듯이 내부 ip로 설정해주면 됩니다.

## 테스트
Slave 서버에서 외부 주소로 ping 이 성공하면 성공입니다.

```
> ping www.google.co.kr
PING www.google.co.kr (58.123.220.104) 56(84) bytes of data.
64 bytes from 58.123.220.104: icmp_seq=1 ttl=56 time=0.859 ms
64 bytes from 58.123.220.104: icmp_seq=2 ttl=56 time=0.961 ms
```